# Easy example

In [337]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from network2tikz import plot


nodes2 = ['a','b','c','d']
edges2 = [('a','b'), ('a','c'), ('c','d'),('d','b')]
gender = ['f', 'm', 'f', 'm']
colors = {'m': 'blue', 'f': 'red'}

style = {}
#style['node_label'] = ['Alice', 'Bob', 'Claire', 'Dennis']
style['node_color'] = [colors[g] for g in gender]
style['node_opacity'] = 1
style['edge_curved'] = .1
style['vertex_size'] = .05
style['vertex_color'] = 'red'

plot((nodes2,edges2),**style) #I cannot make it work with more than ~1500 edges...
plt.show()     

# My code
It does not work with more than 1280 edges... (did a ".head(1500).tail(500)" and it works...)

# EXample with more data and variables

In [ ]:
import igraph
from network2tikz import plot

net = igraph.Graph([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3),
                    (5,6), (6,6)],directed=True)

net.vs["name"] = ["Alice", "Bob", "Claire", "Dennis", "Esther", "Frank", "George"]
net.vs["age"] = [25, 31, 18, 47, 22, 23, 50]
net.vs["gender"] = ["f", "m", "f", "m", "f", "m", "m"]
net.es["is_formal"] = [False, False, True, True, True, False, True, False,
                       False, False]

layout = {0: (4.3191, -3.5352), 1: (0.5292, -0.5292),
          2: (8.6559, -3.8008), 3: (12.4117, -7.5239),
          4: (12.7, -1.7069), 5: (6.0022, -9.0323),
          6: (9.7608, -12.7)}
#plot(net,layout=layout)

color_dict = {'m': 'blue', 'f': 'red'}
visual_style = {}

visual_style['vertex_size'] = .5
visual_style['vertex_color'] = [color_dict[g] for g in net.vs['gender']]
visual_style['vertex_opacity'] = .7
visual_style['vertex_label'] = net.vs['name']
visual_style['vertex_label_position'] = 'below'

visual_style['edge_width'] = [1 + 2 * int(f) for f in net.es('is_formal')]
visual_style['edge_curved'] = 0.1

visual_style['layout'] = layout
visual_style['canvas'] = (8,8)
visual_style['margin'] = 1

plot(net,**visual_style)

In [285]:
import pandas as pd
import numpy as np
import re
import igraph
from network2tikz import plot

In [286]:
dfa = pd.read_csv('/Users/jbg929/Desktop/projects/cath/figure/all_danish_icd10.txt', skiprows=6, sep='\t')
dfa.columns = ['code', 'name']
dfa['ICD10'] = dfa['code'].str[1:]
dfa['ICD8'] = '-1'
dfa.head()

,code,name,ICD10,ICD8
0,DA000,Kolera forårsaget af Vibrio cholerae,A000,-1
1,DA001,Kolera forårsaget af Vibrio cholerae eltor,A001,-1
2,DA009,Kolera UNS,A009,-1
3,DA01,Tyfus og paratyfus,A01,-1
4,DA010,Tyfus,A010,-1


In [287]:
dfm = pd.read_excel('/Users/jbg929/Desktop/projects/cath/figure/danish_mapping.xlsx', dtype=str)
dfm = dfm[['ICD-8', 'ICD-10 match']]
dfm.columns = ['icd8', 'icd10match']
dfm.head()

,icd8,icd10match
0,00009,A000
1,00019,A001
2,00099,A009
3,00199,A010
4,00209,A011


In [288]:
def ch_icd8(code):
    try:
        i = str(code)
        i = i[:3]
    except:
        return -1

    # Chp 1: 
    if (	'000' <= i and i <= '136' ): return 1
    # Chp 2: 
    elif ( '140' <= i and i <= '239' ): return 2
    # Chp 3: 
    elif ( '240' <= i and i <= '279' ): return 3
    # Chp 4: 
    elif ( '280' <= i and i <= '289' ): return 4
    # Chp 5: 
    elif ( '290' <= i and i <= '315' ): return 5
    # Chp 6.1: 
    elif (	'320' <= i and i <= '358' ): return 6
    # Chp 6.2: 
    elif (	'360' <= i and i <= '379' ): return 6
    # Chp 6.3: 
    elif (	'380' <= i and i <= '389' ): return 6
    # Chp 7: 
    elif (	'390' <= i and i <= '458' ): return 7
    # Chp 8: 
    elif (	'460' <= i and i <= '519' ): return 8
    # Chp 9: 
    elif (	'520' <= i and i <= '577' ): return 9
    # Chp 10: 
    elif (	'580' <= i and i <= '629' ): return 10
    # Chp 11: 
    elif (	'630' <= i and i <= '678' ): return 11
    # Chp 12: 
    elif (	'680' <= i and i <= '709' ): return 12
    # Chp 13: 
    elif (	'710' <= i and i <= '738' ): return 13
    # Chp 14: 
    elif (	'740' <= i and i <= '759' ): return 14
    # Chp 15: 
    elif (	'760' <= i and i <= '779' ): return 15
    # Chp 16: 
    elif (	'780' <= i and i <= '796' ): return 16
    # Chp 17: 
    elif (	'800' <= i and i <= '999' ): return 17

    return -1
  
def ch_icd10(code):
	code = str(code)
	if code < 'A': return -1
	if code < 'C00': return 1
	if code < 'D50': return 2
	if code < 'E00': return 3
	if code < 'F00': return 4
	if code < 'G00': return 5
	if code < 'H00': return 6
	if code < 'H60': return 7
	if code < 'I00': return 8
	if code < 'J00': return 9
	if code < 'K00': return 10
	if code < 'L00': return 11
	if code < 'M00': return 12
	if code < 'N00': return 13
	if code < 'O00': return 14
	if code < 'P00': return 15
	if code < 'Q00': return 16
	if code < 'R00': return 17
	if code < 'S00': return 18
	return -1

def get_color(x):
    if x <1: return (220,220,220)
    chapterColors = [None]*22
    chapterColors[1] = (75, 194, 202)
    chapterColors[2] = (158, 79, 70)
    chapterColors[3] = (230, 160, 198)
    chapterColors[4] = (233, 68, 152)
    chapterColors[5] = (75, 183, 72)
    chapterColors[6] = (58, 92, 169)
    chapterColors[7] = (255, 193, 132)
    chapterColors[8] = (169, 166, 210)
    chapterColors[9] = (140, 65, 152)
    chapterColors[10] = (98, 93, 94)
    chapterColors[11] = (8, 132, 149)
    chapterColors[12] = (114, 150, 86)
    chapterColors[13] = (149, 32, 103)
    chapterColors[14] = (248, 223, 28)
    chapterColors[15] = (72, 187, 143)
    chapterColors[16] = (26, 97, 49)
    chapterColors[17] = (245, 132, 31)
    chapterColors[18] = (38, 40, 111)
    chapterColors[19] = (237, 34, 36)
    chapterColors[20] = (183, 200, 73)
    chapterColors[21] = (2, 2, 3)
    return chapterColors[x]

In [289]:
#Putting all ICD10 together
df = pd.concat([dfm, dfa.rename(columns={'ICD8':'icd8', 'ICD10':'icd10match'})[['icd8', 'icd10match']]])

#removing anything that is not alphanumeric
#THese should be numbers!! df['icd8'] = df['icd8'].apply(lambda x: re.sub(r'\W+', '', str(x)))
df['icd10match'] = df['icd10match'].apply(lambda x: re.sub(r'\W+', '', str(x)))

#Removing duplicates
#df.drop_duplicates('icd8', keep='first', inplace=True)

df = df.sort_values('icd8', ascending=False)

df['icd8'] = '00000'+df['icd8'].astype(str)
df['icd8'] = df['icd8'].str[-5:]

df.tail()

,icd8,icd10match
6895,000-1,I409
6894,000-1,I408A
6893,000-1,I408
6892,000-1,I401
20691,000-1,UM09


In [290]:
df['icd10match'] = np.where(df['icd10match'] == 'nan', '000-1', df['icd10match'])

#Getting the chapter for each code (icd8 and icd10)
df['ch_i8'] = df['icd8'].apply(ch_icd8)
df['ch_i10'] = df['icd10match'].apply(ch_icd10)

df['color'] = df['ch_i10'].apply(get_color)

df['ch_i8'] = df['ch_i8'].astype(str)
df['ch_i10'] = df['ch_i10'].astype(str)

df.sort_values(['icd8','icd10match'], ascending=True)

df.head()

,icd8,icd10match,ch_i8,ch_i10,color
10422,Y9589,000-1,-1,-1,"(220, 220, 220)"
10421,Y9549,000-1,-1,-1,"(220, 220, 220)"
10420,Y9539,000-1,-1,-1,"(220, 220, 220)"
10419,Y9529,000-1,-1,-1,"(220, 220, 220)"
10418,Y9519,000-1,-1,-1,"(220, 220, 220)"


In [291]:
nodes = list(set(df['icd8'].tolist()+df['icd10match'].tolist()))
nodes = np.array(nodes)

#getting the node ID
df['nid8'] = df['icd8'].apply(lambda x: np.where(nodes == x)[0][0])
df['nid10'] = df['icd10match'].apply(lambda x: np.where(nodes == x)[0][0])
df.head()

,icd8,icd10match,ch_i8,ch_i10,color,nid8,nid10
10422,Y9589,000-1,-1,-1,"(220, 220, 220)",18693,2361
10421,Y9549,000-1,-1,-1,"(220, 220, 220)",20955,2361
10420,Y9539,000-1,-1,-1,"(220, 220, 220)",28220,2361
10419,Y9529,000-1,-1,-1,"(220, 220, 220)",6872,2361
10418,Y9519,000-1,-1,-1,"(220, 220, 220)",27553,2361


In [292]:
#creating the positions

#icd8
a = df.drop_duplicates('nid8', keep='first').sort_values('icd8')
a = a.reset_index(drop=True).reset_index().rename(columns={'index':'y', 'nid8':'nid'})[['y','nid']]
a['x'] = 1

#icd10
b = df.drop_duplicates('nid10', keep='first').sort_values('nid10')
b = b.reset_index(drop=True).reset_index().rename(columns={'index':'y', 'nid10':'nid'})[['y','nid']]
b['x'] = 7

concat = pd.concat([a,b])
concat = concat.astype(str)
concat = concat.apply(lambda row: row['nid']+':('+row['x']+','+row['y']+')', axis=1).values
positions = eval('{'+','.join(concat)+'}')

In [293]:
#Getting the edges
edges = list(df.apply(lambda row: (row['nid8'],row['nid10']), axis=1).values)

In [294]:
import igraph
from network2tikz import plot

net = igraph.Graph(edges,directed=True)

net.vs["name"] = nodes
net.vs["chapter10"] = df['ch_i10'].tolist() #TODO: This may be a mistake
net.es["chapter_edge"] = df['ch_i10'].tolist()

layout = positions


color_dict = {'-1':(220,220,220),
    '1':(75, 194, 202),
    '2':(158, 79, 70),
    '3':(230, 160, 198),
    '4':(233, 68, 152),
    '5':(75, 183, 72),
    '6':(58, 92, 169),
    '7':(255, 193, 132),
    '8':(169, 166, 210),
    '9':(140, 65, 152),
    '10':(98, 93, 94),
    '11':(8, 132, 149),
    '12':(114, 150, 86),
    '13':(149, 32, 103),
    '14':(248, 223, 28),
    '15':(72, 187, 143),
    '16':(26, 97, 49),
    '17':(245, 132, 31),
    '18':(38, 40, 111),
    '19':(237, 34, 36),
    '20':(183, 200, 73),
    '21':(2, 2, 3)}

visual_style = {}

visual_style['vertex_size'] = .5
visual_style['vertex_color'] = [color_dict[g] for g in net.vs['chapter10']]
visual_style['vertex_opacity'] = .7
visual_style['vertex_label'] = net.vs['name']
visual_style['vertex_label_position'] = 'below'

visual_style['edge_width'] = [2]*len(edges)
visual_style['edge_curved'] = 0.1
#visual_style['edge_color'] = [color_dict[g] for g in net.es["chapter_edge"]]

visual_style['layout'] = layout
visual_style['canvas'] = (8,8)
visual_style['margin'] = 1

plot(net,**visual_style)

No LaTex compiler was found! Either specify a LaTex compiler or make sure you have latexmk or pdfLaTex installed.


CnetError: 